In [ ]:
!gdown  1bSBMkhXFZLEdZtCAvSexol4d3bXz6wR9

In [ ]:
import re

In [ ]:
#!head -n100000 manchester_input_tagged_trf.txt > test_text

In [ ]:
file="manchester_input_tagged_trf.txt"
#file="test_text"
output="manchester_input_tagged_trf_postprocessed.txt"
with open(output, 'w') as fi:
    with open(file, 'r') as infile:
        lines = infile.readlines()
        i = 0
        for line in lines:
            i = i+1
            print(i)
            line = line.rstrip()
            line = re.sub("_PROPN","_NOUN",line)
            line = re.sub("gon_([^ ]+) na_([^ ]+)","gonna_VERB",line)
            line = re.sub("got_([^ ]+) ta_([^ ]+)","gotta_VERB",line)
            line = re.sub("([^ ]+)_AUX","\\1_VERB",line)
            line = re.sub("([^ ]+)'ll_([^ ]+)","\\1_NOUN 'll_VERB",line)
            line = re.sub("([a-z])@l_([^ ]+)","\\1@l_NOUN",line)
            line = re.sub("([^ ]+)_NOUN 's_PART","\\1's_NOUN",line)
            line = re.sub("([^ ]+)_PRON 's_PART","\\1's_PRON",line)
            # We extracted a list of conjoined elements that are tagged as NOUN.
            # The following items were judged not to be NOUNs and so are retagged.
            line = re.sub("night_night_NOUN","night_night_X",line)
            line = re.sub("a_lot_of_NOUN","a_lot_of_X",line)
            line = re.sub("lots_of_NOUN","lots_of_X",line)
            line = re.sub("happy_birthday_NOUN","happy_birthday_X",line)
            line = re.sub("as_well_NOUN","as_well_X",line)
            line = re.sub("tickle_tickle_NOUN","tickle_tickle_X",line)
            line = re.sub("see_saw_marjorie_daw_NOUN","see_saw_marjorie_daw_X",line)
            line = re.sub("thank_you_NOUN","thank_you_X",line)
            line = re.sub("wakie_wakie_NOUN","wakie_wakie_X",line)
            line = re.sub("so_that_NOUN","so_that_X",line)
            line = re.sub("as_well_as_NOUN","as_well_as_X",line)
            # We extracted all items that are tagged as NOUN. We checked all items that occured more than 20 times (0.01%).
            # Any items that we thought not to be NOUNS were checked.
            # The following items were judged not to be NOUNs after checking and so are retagged as X.
            line = re.sub("(sposta)_NOUN","\\1_X")
            line = re.sub("(hafta)_NOUN","\\1_X",line)
            line = re.sub("(o'clock)_NOUN","\\1_X",line)
            line = re.sub("(\\'s)_NOUN","\\1_X",line)
            line = re.sub("(none)_NOUN","\\1_X",line)
            line = re.sub("(hasta)_NOUN","\\1_X",line)
            line = re.sub("(pretend)_NOUN","\\1_X",line)
            line = re.sub("(-)_NOUN","\\1_X",line)
            line = re.sub("(useta)_NOUN","\\1_X",line)
            line = re.sub("(yours)_NOUN","\\1_X",line)
            line = re.sub("(upsidedown)_NOUN","\\1_X",line)

            #all WORD'll -> WORD_NOUN 'll_VERB

            output += line
            output = output + "\n"
        fi.write(output)

AUX -> VERB
PropN -> Noun
gon na -> gonna (need to be VERBs)
got ta -> gotta (need to be VERBs)
all WORD'll -> WORD_NOUN 'll_VERB (currently only does pronoun)

all @l elements should be tagged as NOUN

possessive s is split - we want to put it back together.

Find conjoined elements that are incorrectly tagged as NOUN and retag as X

Correct things that are consistently misclassified as NOUN or VERB



[Could also check whether item that are sometimes classified as NOUNs and VERBs are also sometimes tagged as anything else. Currently don't do as Brusini et al didn't.]


In [ ]:
!grep  " use_NOUN" manchester_input_tagged_trf_postprocessed.txt

In [ ]:
import re
from collections import defaultdict
noun_tokens=defaultdict(int)
verb_tokens=defaultdict(int)
#tokens_tags=dict()
tokens=[]
tags=[]
filename="manchester_input_tagged_trf_postprocessed.txt"
with open(filename) as file:
        for line in file:
            tokens.append("{")
            tags.append("BOS")
            line_array = line.split()
            for element in line_array:
                la=re.match("([^ ]+)\\_([^ ]+)",element)
                w=la.group(1)
                t=la.group(2)
                tokens.append(w)
                tags.append(t)
                if re.match("NOUN",t):
                    noun_tokens[str.lower(w)] += 1
                if re.match("VERB",t):
                    verb_tokens[str.lower(w)] += 1
            tokens.append("}")
            tags.append("EOS")


In [ ]:
!grep "as_well_as_NOUN" manchester_input_tagged_trf_postprocessed.txt

In [ ]:
sorted_noun_counts=sorted(noun_tokens.items(), key=lambda item: item[1], reverse=True)
sorted_verb_counts=sorted(verb_tokens.items(), key=lambda item: item[1], reverse=True)
# Extract the list of tokens, by transposing the list of lists so that there is a list of tokens a list of counts and then just selecting the former
sorted_noun_tokens=list(zip(*sorted_noun_counts))[0]
sorted_verb_tokens=list(zip(*sorted_verb_counts))[0]
excluded_nouns=["mummy","daddy","john","carl","dominic"] # proper names
excluded_verbs=[""]
sorted_noun_tokens=[x for x in sorted_noun_tokens if x not in excluded_nouns]
sorted_noun_tokens=[x for x in sorted_noun_tokens if noun_tokens[x] > verb_tokens[x]]
sorted_verb_tokens=[x for x in sorted_verb_tokens if x not in excluded_verbs]
sorted_verb_tokens=[x for x in sorted_verb_tokens if verb_tokens[x] > noun_tokens[x]]
seed_set_size = 10
noun_seeds=sorted_noun_tokens[0:seed_set_size]
verb_seeds=sorted_verb_tokens[0:seed_set_size]
type_list=list(noun_seeds)
type_list.extend(verb_seeds)
tokens.insert(0,"{")
tokens.insert(len(tokens),"}")
token_count=len(tokens)

In [ ]:
import pandas as pd
sorted_noun_counts=sorted(noun_tokens.items(), key=lambda item: item[1], reverse=True)
sorted_noun_tokens=list(zip(*sorted_noun_counts))[0]


In [ ]:
compounds=pd.DataFrame([[re.sub("[\\+_]"," ",i[0]) for i in sorted_noun_counts if re.search("[\\+\\_]",i[0])],[i[1] for i in sorted_noun_counts if re.search("[\\+\\_]",i[0])]]).transpose()

In [ ]:
compounds

In [ ]:
compounds.to_csv("compounds.csv")

In [ ]:
import pandas as pd

sorted_noun_counts=pd.DataFrame(data=sorted_noun_counts)
sorted_verb_counts=pd.DataFrame(data=sorted_verb_counts)
sorted_noun_counts.to_csv("nouns_for_checking.csv")
sorted_verb_counts.to_csv("verbs_for_checking.csv")

problem words:

one

bit

pardon